In [1]:
import pandas as pd
import numpy as np
import gensim
from helpers import lemmatize_stemming, get_preprocessed, compute_coherence_values

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline


filenames = !ls data/unzipped
filenames = [ 'data/unzipped/'+filename for filename in filenames ]

import random
# indeces = random.sample(range(len(filenames)), 50)
indeces = [0]

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/pylab/config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):
/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
from helpers import load_tweets

documents = load_tweets(np.array(filenames)[indeces])

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
processed_docs = [ get_preprocessed(doc) for doc in documents ]
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=14, id2word=dictionary, passes=10, workers=4, alpha=0.01, eta=.91)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Topic: 0 
Words: 0.012*"lie" + 0.010*"tell" + 0.010*"trump" + 0.010*"time" + 0.009*"good" + 0.007*"respons" + 0.007*"read" + 0.007*"realdonaldtrump" + 0.007*"coronavirus" + 0.005*"piec"
Topic: 1 
Words: 0.014*"think" + 0.008*"virus" + 0.007*"coronavirus" + 0.006*"pandem" + 0.006*"news" + 0.006*"fake" + 0.006*"johnson" + 0.006*"drop" + 0.005*"treatment" + 0.005*"peak"
Topic: 2 
Words: 0.019*"trump" + 0.016*"evid" + 0.013*"see" + 0.010*"coronavirus" + 0.010*"wuhan" + 0.009*"china" + 0.009*"say" + 0.008*"chines" + 0.008*"origin" + 0.007*"suggest"
Topic: 3 
Words: 0.020*"covid" + 0.010*"coronavirus" + 0.007*"thank" +

In [ ]:
lda_model.save('trained_lda')

In [ ]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus_tfidf, dictionary)
vis

In [ ]:
len(documents)